# General preparations

In [1]:
from misc.misc import run_in_thread_simple
%load_ext autoreload
%autoreload 2
# %matplotlib notebook
import matplotlib
matplotlib.use('TKAgg')
import os 
import datetime
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_modes2 import gen_input_spots_array, gen_output_modes_Unitary
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
import time 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_09_QKD_random_phases'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

In [3]:
# MPLC
from pianoq.lab.mplc.mplc_device import MPLCDevice
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

Got MPLC


In [5]:
# get masks
import glob 
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
paths = glob.glob(fr'{dir_path}\*.masks')
assert len(paths) == 1, 'there should be one .masks file'
masks_path = paths[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
orig_masks = msks.real_masks

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

## Checklist
- [X] make list
- [X] create masks WFM
- [X] measure single counts 
- [X] mark spots locations 
- [ ] find phases
- [ ] measure coincidence no phases
- [ ] measure with good phases 
- [ ] measure with random phases X 5 

# Create masks WFM

In [9]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path = fr'{dir_path}\{timestamp}_QKD_MUB2.masks'
mplc_sim = create_WFM_QKD_masks(N_iterations=15, out_path=masks_path)

100%|██████████| 15/15 [03:12<00:00, 12.84s/it]


In [10]:
mplc_sim.res._calc_normalized_overlap()
display(np.angle(np.diag(mplc_sim.res.forward_overlap)))
display(np.diag(mplc_sim.res.forward_overlap))
mplc_sim.res._calc_fidelity()
display(mplc_sim.res.fidelity)

array([-0.01104283, -0.01694016, -0.01549735, -0.00316909, -0.00069085,
        0.0036713 ,  0.02134649,  0.01103632, -0.01880339,  0.00477507],
      dtype=float32)

array([0.6407918 -0.00707644j, 0.6609486 -0.01119764j,
       0.6609756 -0.01024419j, 0.63431716-0.00201022j,
       0.6449323 -0.00044555j, 0.63141215+0.00231811j,
       0.6488427 +0.01385262j, 0.6300257 +0.00695345j,
       0.6547122 -0.01231226j, 0.64000374+0.00305609j], dtype=complex64)

0.9859997292423204

In [11]:
mplc_sim.res.show_overlap()

In [13]:
fig, axes = plt.subplots(2, 5)
for i, ax in enumerate(axes.flat):
    # ax.imshow(np.angle(res.masks[i][360:720, 140:280]), cmap='gray')
    ax.imshow(np.angle(mplc_sim.res.masks[i][360:720, 140:280]), cmap='gray')

In [15]:
mplc_sim.res.show_all(7)

# Find locations with single counts 

In [6]:
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan
import numpy as np
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult

masks = orig_masks.copy()
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

In [9]:
resolution = 1
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal_res_{resolution}.scan'
singles_signal_path = fr'{dir_path}\{timestamp}_QKD_MUB2_idler_res_{resolution}.scan'

## perform scan

In [10]:
t = run_in_thread(signal_scan, integration_time=1.0, coin_window=2e-9, resolution=resolution, out_path=singles_signal_path, half_scan=True)
t = run_in_thread(idler_scan, integration_time=1.0, coin_window=2e-9, resolution=resolution, out_path=singles_idler_path, half_scan=True)

## mark spots locations  

In [6]:
from pianoq_results.scan_result import ScanResult

path_sig = glob.glob(fr'{dir_path}\*signal_res_1.scan')[0]
path_idl = glob.glob(fr'{dir_path}\*idler_res_1.scan')[0]

res_sig = ScanResult()
res_idl = ScanResult()
res_idl.loadfrom(path_idl)
res_sig.loadfrom(path_sig)
res_idl.show_singles()
res_sig.show_singles()

In [18]:
_ = res_sig.get_xys(1, saveto_dir=dir_path)
_ = res_idl.get_xys(2, saveto_dir=dir_path)

Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_05_QKD_random_phases\2024_09_09_09_22_41_s1_idl.locs


# Find phases

In [ ]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder
import glob 
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan
import numpy as np
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import time 


locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

masks = orig_masks.copy()
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)

mplc.load_masks(masks, linear_tilts=True)
initiail_phases = np.zeros(25 * 2)

phases_path = fr'{dir_path}\{timestamp}_QKD_MUB2_signal.phases'

# i,j of corr matrix that is supposed to be strong 
i = 0
j = 0
pf = PhaseFinder(mplc=m, integration_time=30, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=initiail_phases, coin_window=2e-9)
pf.m_idl_x.move_absolute(locs_idl[i, 0])
pf.m_idl_y.move_absolute(locs_idl[i, 1])
pf.m_sig_x.move_absolute(locs_sig[j, 0])
pf.m_sig_y.move_absolute(locs_sig[j, 1])

In [4]:
initiail_phases = np.zeros(25 * 2)
pf.res.phases = initiail_phases
pf.res.initial_phases = initiail_phases

m.restore_location()

In [5]:
pf.find_phases()

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
3,0
3,1
3,2
3,3
3,4
3,5
3,6
3,7
3,8
3,9
4,0
4,1
4,2
4,3
4,4
4,5
4,6
4,7
4,8
4,9
5,0
5,1
5,2
5,3
5,4
5,5
5,6
5,7
5,8
5,9
6,0
6,1
6,2
6,3
6,4
6,5
6,6
6,7
6,8
6,9
7,0
7,1
7,2
7,3
7,4
7,5
7,6
7,7
7,8
7,9
8,0
8,1
8,2
8,3
8,4
8,5
8,6
8,7
8,8
8,9
9,0
9,1
9,2
9,3
9,4
9,5
9,6
9,7
9,8
9,9


In [6]:
pf.close()

In [7]:
pf.res.plot_best_phases()

3
8
13
18
23
28
33
38
43
48


# Scan correlations 

In [5]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.mplc_device import MPLCDevice
import glob

masks = orig_masks.copy()
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)

phases_result = PhaseFinderResult()
phases_path = glob.glob(fr'{dir_path}\*.phases')[0]
phases_result.loadfrom(phases_path)

# Important part here! <--------------------------------------
phases_type = 'good'
# phases_type = 'none'
# phases_type = 'random'
if phases_type == 'good':
    phases = phases_result.phases
elif phases_type == 'random':
    phases = np.random.uniform(0, 2*np.pi, len(phases_result.phases))
elif phases_type == 'none':
    phases = np.zeros_like(phases_result.phases)
else:
    raise ValueError(f'phases type {phases_type} not recognized')
    
masks = add_phase_input_spots(masks, phases)

locs_sig_path = glob.glob(fr'{dir_path}\*sig.locs')[0]
locs_idl_path = glob.glob(fr'{dir_path}\*idl.locs')[0]
locs_sig = np.load(locs_sig_path)['locs']
locs_idl = np.load(locs_idl_path)['locs']

mplc = MPLCDevice()
mplc.load_masks(masks, linear_tilts=True)

backlash = 0.0
wait_after_move = 0.3
integration_time = 4
coin_window = 2e-9
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s.scan'
dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window)


Got Zaber motors!
Got Thorlabs motors!
Got TimeTagger!


In [15]:
mplc.restore_location()

In [6]:
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
scan_path = fr'{dir_path}\{timestamp}_QKD_MUB2_{phases_type}_exp_{integration_time}s.scan'
dps.scan()

beginning scan
0, 0: 4880.50, 6002.25, 4.00
0, 1: 4801.50, 5984.75, 0.50
0, 2: 4721.25, 5253.50, 0.00
0, 3: 4796.25, 6469.75, 0.75
0, 4: 4793.00, 5994.00, 0.25
1, 0: 5302.50, 5951.75, 0.25
1, 1: 5296.75, 6019.75, 5.00
1, 2: 5272.25, 5252.50, 0.50
1, 3: 5291.75, 6454.50, 1.00
1, 4: 5313.75, 6000.50, 0.00
2, 0: 4637.75, 5927.75, 0.00
2, 1: 4622.00, 6031.25, 0.00
2, 2: 4545.25, 5240.00, 2.25
2, 3: 4597.25, 6415.75, 1.00
2, 4: 4610.50, 6032.50, 0.50
3, 0: 5042.75, 5968.50, 0.00
3, 1: 5073.00, 5929.00, 0.25
3, 2: 5082.00, 5283.00, 0.50
3, 3: 5028.00, 6448.50, 3.50
3, 4: 5036.75, 6046.50, 0.50
4, 0: 4376.00, 5890.50, 0.50
4, 1: 4417.00, 5917.75, 0.25
4, 2: 4392.00, 5265.25, 0.00
4, 3: 4388.25, 6535.75, 0.50
4, 4: 4472.25, 6005.25, 1.75


In [12]:
dps.close()
mplc.close()

In [7]:
dps.res.show()
dps.res.show_singles()